In [22]:
# Imports necessários

import kagglehub
import pandas as pd
import os
from ydata_profiling import ProfileReport
import scipy.stats as stats


In [23]:
path = kagglehub.dataset_download("piterfm/paris-2024-olympic-summer-games")
print("Path to dataset files:", path)

Path to dataset files: /Users/brittope/.cache/kagglehub/datasets/piterfm/paris-2024-olympic-summer-games/versions/27


In [24]:
medals_total_df = pd.read_csv(os.path.join(path, "medals_total.csv"))
medallists_df = pd.read_csv(os.path.join(path, "medallists.csv"))
athletes_df = pd.read_csv(os.path.join(path, "athletes.csv"))

In [25]:
profile = ProfileReport(medals_total_df, title="Relatório - Total de Medalhas")
profile.to_file("medals_total_report.html")
profile = ProfileReport(medallists_df, title="Relatório - Medalhistas")
profile.to_file("medallists_report.html")
profile = ProfileReport(athletes_df, title="Relatório - Atletas")
profile.to_file("athletes_report.html")
print("Reports saved as medals_total_report.html and medallists_report.html and athletes_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 146.74it/s]

Reports saved as medals_total_report.html and medallists_report.html and athletes_report.html


##### 1. Quantos países ganharam pelo menos uma medalha?

    92 países ganharam pelo menos uma medalha.

##### 2. Quantas medalhas foram distribuídas?

    Ao total 2315 medalhas foram distribuídas.

In [26]:
filtered_athletes_df = athletes_df[athletes_df.height != 0]
filtered_athletes_df.groupby('gender')['height'].mean()

gender
Female    172.224828
Male      185.363916
Name: height, dtype: float64

#### 3. Qual a altura média, respectivamente, dos atletas homens e mulheres?

    Para homens a média foi de 185.3 cm de altura, já para mulheres a média foi de 172.2 cm de altura.

#### 4. Na base de atletas, qual campo tem mais valores faltantes, entre gênero, apelido, país de residência, país de nascimento?

    O campo com mais valores faltantes é o de apelido.

In [27]:
medallists_df['medal_date'].mode()

0    2024-08-10
Name: medal_date, dtype: object

#### 5. Qual dia foi distribuído o maior número de medalhas?

    No dia 10/08/2024 foi distribuído o maior número de medalhas em um único dia.

In [28]:
medallists_df['discipline'].mode()

0    Athletics
Name: discipline, dtype: object

#### 6. Qual esporte foi distribuído o maior número de medalhas?

    O esporte que destribuíu o maior número de medalhas foi o atletismo.

In [29]:
medallists_df['name'].mode()

0    ZHANG Yufei
Name: name, dtype: object

#### 7. Qual dessas afirmações é verdadeira?
*   Yufei Zhang é o atleta que mais ganhou medalhas.
*   O número de homens que participaram é inferior ao de mulheres.
*   O número de medalhas de ouro, prata e bronze é igual.
*   Pelo menos metade dos países participantes ganhou pelo menos uma medalha.

        A afirmativa correta é a primeira, Yufei Zhang é o atleta que mais ganhou medalhas nestes jogos olimpicos.

In [30]:
populacao_df = pd.read_csv("../DataFrames/population.csv", skiprows=3)
populacao_df = populacao_df[['Country Name', 'Country Code', '2023']]
populacao_df.rename(columns={'2023': 'population'}, inplace=True)

medalhas_e_populacao_df = pd.merge(medals_total_df, populacao_df, left_on='country_code', right_on='Country Code')

renda_per_capita_df = pd.read_csv("../DataFrames/GDP.csv", skiprows=3)
renda_per_capita_df = renda_per_capita_df[['Country Name', 'Country Code', '2023']]
renda_per_capita_df.rename(columns={'2023': 'gdp'}, inplace=True)

medalhas_populacao_e_gdp_df = pd.merge(medalhas_e_populacao_df, renda_per_capita_df, left_on='country_code', right_on='Country Code')

medalhas_populacao_e_gdp_df['gdp_per_capita'] = medalhas_populacao_e_gdp_df['gdp'] / medalhas_populacao_e_gdp_df['population']

area_total_df = pd.read_csv("../dataFrames/totalArea.csv", skiprows=3)
area_total_df = area_total_df[['Country Name', 'Country Code', '2022']]
area_total_df.rename(columns={'2022': 'area'}, inplace=True)

medalhas_populacao_gdp_e_area_df = pd.merge(medalhas_populacao_e_gdp_df, area_total_df, left_on='country_code', right_on='Country Code')

medalhas_populacao_gdp_e_area_df = medalhas_populacao_gdp_e_area_df[['country_code','country', 'Gold Medal', 'Silver Medal', 'Bronze Medal', 'Total', 'population', 'gdp', 'gdp_per_capita', 'area']]

medalhas_populacao_gdp_e_area_df['medal_per_population'] = medalhas_populacao_gdp_e_area_df['population'] / medalhas_populacao_gdp_e_area_df['Total']
#medalhas_populacao_gdp_e_area_df = medalhas_populacao_gdp_e_area_df[medalhas_populacao_gdp_e_area_df['Total'] >= 10]
medalhas_populacao_gdp_e_area_df.sort_values(by='medal_per_population', ascending=True)

,country_code,country,Gold Medal,Silver Medal,Bronze Medal,Total,population,gdp,gdp_per_capita,area,medal_per_population
44,DMA,Dominica,1,0,0,1,6.651000e+04,6.539926e+08,9832.996431,750.0,6.651000e+04
39,CHI,Chile,1,1,0,2,1.676910e+05,1.250793e+10,74589.138023,NaN,8.384550e+04
40,LCA,Saint Lucia,1,1,0,2,1.792850e+05,2.430148e+09,13554.665188,620.0,8.964250e+04
26,BRN,Bahrain,2,1,1,4,4.589490e+05,1.512829e+10,32962.906512,5770.0,1.147372e+05
8,NZL,New Zealand,10,7,3,20,5.223100e+06,2.521755e+11,48280.811417,267710.0,2.611550e+05
...,...,...,...,...,...,...,...,...,...,...,...
34,ETH,Ethiopia,1,3,0,4,1.286917e+08,1.636979e+11,1272.016282,1136239.7,3.217292e+07
62,PER,Peru,0,0,1,1,3.384562e+07,2.676032e+11,7906.585029,1285220.0,3.384562e+07
37,EGY,Egypt,1,1,1,3,1.145358e+08,3.960025e+11,3457.456916,1001450.0,3.817859e+07
53,IND,India,0,1,5,6,1.438070e+09,3.567552e+12,2480.792087,3287260.0,2.396783e+08


In [31]:
medalhas_populacao_gdp_e_area_df.dropna(subset=['population'], inplace=True)
correlation, p_value = stats.pearsonr(medalhas_populacao_gdp_e_area_df['population'], medalhas_populacao_gdp_e_area_df['Total'])
print(f"Correlação população: {correlation:.2f}")
print("P-value:", p_value)
print("\n")
medalhas_populacao_gdp_e_area_df.dropna(subset=['gdp'], inplace=True)
correlation, p_value = stats.pearsonr(medalhas_populacao_gdp_e_area_df['gdp'], medalhas_populacao_gdp_e_area_df['Total'])
print(f"Correlação gdp: {correlation:.2f}")
print("P-value:", p_value)
print("\n")
medalhas_populacao_gdp_e_area_df.dropna(subset=['area'], inplace=True)
correlation, p_value = stats.pearsonr(medalhas_populacao_gdp_e_area_df['area'], medalhas_populacao_gdp_e_area_df['Total'])
print(f"Correlação area: {correlation:.2f}")
print("P-value:", p_value)
print("\n")
medalhas_populacao_gdp_e_area_df.dropna(subset=['gdp_per_capita'], inplace=True)
correlation, p_value = stats.pearsonr(medalhas_populacao_gdp_e_area_df['gdp_per_capita'], medalhas_populacao_gdp_e_area_df['Total'])
print(f"Correlação gdp per capita: {correlation:.2f}")
print("P-value:", p_value)

Correlação população: 0.40
P-value: 0.0010274212931665702


Correlação gdp: 0.87
P-value: 2.803665197306508e-20


Correlação area: 0.56
P-value: 2.4231632831816926e-06


Correlação gdp per capita: 0.37
P-value: 0.002916745400798887


In [32]:
medalhas_populacao_gdp_e_area_df['medal_per_population'].mean()

np.float64(14216978.67242368)